In [9]:
import numpy as np
import pandas as pd 
import sklearn

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

In [3]:
data=pd.read_csv('toronto_data2.csv')

In [5]:
toronto_data=data

In [6]:
# Combining the text in Keywords and categories columns
toronto_data['All_Keywords'] = toronto_data['categories'].str.cat(toronto_data['Keywords'],sep=", ")

# Formating the All_Keywords Column
toronto_data['All_Keywords'] = toronto_data['All_Keywords'].map(lambda x: str(x))
toronto_data['All_Keywords'] = toronto_data['All_Keywords'].map(lambda x: x.lower())

# Adding and Grouping Rows together by Restaurant Name
toronto_final = toronto_data.groupby('name')['All_Keywords'].sum()
toronto_final = toronto_final.to_frame(name = 'sum').reset_index()

# Getting a list of Unique Keywords per Restaurant

toronto_final['sum'] = toronto_final['sum'].map(lambda x: x.replace(", ","', '"))
toronto_final['sum'] = toronto_final['sum'].map(lambda x: str("'") + x + str("'"))
f = lambda x: x["sum"].split(", ")
toronto_final['sum'] = toronto_final.apply(f, axis=1)
toronto_final['sum'] = toronto_final['sum'].map(lambda x: set(x))
toronto_final.set_index('name', inplace = True)

# Creating Bag of Words
toronto_final['bag_of_words'] = ''
columns = toronto_final.columns
for index, row in toronto_final.iterrows():
    words = ''
    for col in columns:
            words = words + ' '.join(row[col])+ ' '
    row['bag_of_words'] = words
    
toronto_final.drop(columns = [col for col in toronto_final.columns if col!= 'bag_of_words'], inplace = True)

# Remove quotation marks
toronto_final['bag_of_words'] = toronto_final['bag_of_words'].map(lambda x: x.replace("'", ""))

In [13]:
toronto_final.to_csv('bagofWord.csv')

In [10]:
count = CountVectorizer()
count_matrix = count.fit_transform(toronto_final['bag_of_words'])

In [11]:
count_matrix

<3410x1645 sparse matrix of type '<class 'numpy.int64'>'
	with 146287 stored elements in Compressed Sparse Row format>

In [12]:
indices = pd.Series(toronto_final.index)

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
indices

0       10 Barrel Brewing Portland
1                  108 Bar N Grill
2         10th Avenue Liquor Store
3               11th Avenue Liquor
4               13 Virtues Brewing
                   ...            
3405                          kBap
3406           krazy cups and cafe
3407                    red e café
3408                      show bar
3409                   veggie bowl
Name: name, Length: 3410, dtype: object

In [ ]:
indices

In [45]:
data2

,Unnamed: 0,business_id,name,city,state,stars,review_count,categories,latitude,longitude,is_open,postal_code,key
0,1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,Portland,OR,4.0,126,"Salad, Soup, Sandwiches, Delis, Restaurants, C...",45.588906,-122.593331,1,97218,tCbdrRPZA0oiIYSmHG3J0w
1,36,jx91IMdGOmLOo8h_F9z39g,Cleary's Restaurant & Spirits,Portland,OR,3.5,19,"Nightlife, Sandwiches, Seafood, Restaurants",45.526473,-122.535323,1,97230,jx91IMdGOmLOo8h_F9z39g
2,50,5PsApTz-WqhF_-8mfqGtGw,Cafe Yumm!,Portland,OR,3.5,158,"Vegetarian, Vegan, Sandwiches, Soup, American ...",45.518450,-122.675723,1,97204,5PsApTz-WqhF_-8mfqGtGw
3,91,Npm0cjoyWwyV13OULL9qOA,Nana's Ice Cream Scoop Shop,Portland,OR,4.0,5,"Food, Ice Cream & Frozen Yogurt",45.560921,-122.662447,1,97211,Npm0cjoyWwyV13OULL9qOA
4,100,CBkF_Ub_nxOhOEN3Zbo_RQ,Plaid Pantry,Portland,OR,4.0,10,"Convenience Stores, Food",45.562430,-122.687739,1,97217,CBkF_Ub_nxOhOEN3Zbo_RQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4105,160507,2JKien1H998FluEYs0xIrg,Buranko,Portland,OR,5.0,26,"Cocktail Bars, Asian Fusion, Restaurants, Bars...",45.520940,-122.672200,1,97204,2JKien1H998FluEYs0xIrg
4106,160512,7r311z2Uq_UzHqu5hiipxA,Thai Garden,Portland,OR,4.0,165,"Food Stands, Food, Street Vendors, Restaurants...",45.519297,-122.558636,1,97233,7r311z2Uq_UzHqu5hiipxA
4107,160521,68nXXzFyK8tzcBlv8eM3tw,McMenamins Barley Mill Pub,Portland,OR,3.5,108,"Pubs, Breweries, Food, American (Traditional),...",45.512422,-122.648782,1,97214,68nXXzFyK8tzcBlv8eM3tw
4108,160561,jYgqSazE0gUyI7qq086Dzw,Chart House,Portland,OR,3.5,457,"Event Planning & Services, Steakhouses, Seafoo...",45.483154,-122.682748,1,97239,jYgqSazE0gUyI7qq086Dzw


In [46]:
a=data2[data2['name']=="Let's Roll PDX - Division"]
a.append(a)

,Unnamed: 0,business_id,name,city,state,stars,review_count,categories,latitude,longitude,is_open,postal_code,key
2453,93750,MZeXTr-0OELr9e6Vu9h71g,Let's Roll PDX - Division,Portland,OR,4.5,9,"Sushi Bars, Asian Fusion, Restaurants, Food, Poke",45.504613,-122.6256,1,97202,MZeXTr-0OELr9e6Vu9h71g
2453,93750,MZeXTr-0OELr9e6Vu9h71g,Let's Roll PDX - Division,Portland,OR,4.5,9,"Sushi Bars, Asian Fusion, Restaurants, Food, Poke",45.504613,-122.6256,1,97202,MZeXTr-0OELr9e6Vu9h71g


In [49]:
data1=data2

In [47]:
name2=['Momiji Sushi Bar & Restaurant', "Let's Roll PDX - Division", 'Poke Box Sushi Bar', 'Teppanyaki Hut - North Portland', 'Krispy Rice', 'Meeka Japanese Restaurant', 'SanSai Japanese Grill', 'Hissho Sushi', 'Fujiyama Sushi Bar', 'Bluefin Sushi Bar']

In [62]:
df_output2=data1[data1['name']==name2[0]]
for i in name2[1:]:
    df_output2=df_output2.append(data1[data1['name']==i], ignore_index=True)

In [63]:
df_output2

,Unnamed: 0,business_id,name,city,state,stars,review_count,categories,latitude,longitude,is_open,postal_code,key
0,42400,u8_RnVT1ntjkKiPHcV0ixg,Momiji Sushi Bar & Restaurant,Portland,OR,4.0,11,"Sushi Bars, Japanese, Restaurants",45.548491,-122.617841,1,97213,u8_RnVT1ntjkKiPHcV0ixg
1,93750,MZeXTr-0OELr9e6Vu9h71g,Let's Roll PDX - Division,Portland,OR,4.5,9,"Sushi Bars, Asian Fusion, Restaurants, Food, Poke",45.504613,-122.625600,1,97202,MZeXTr-0OELr9e6Vu9h71g
2,92413,b8wixAfPZpMs221Y-HBVLA,Poke Box Sushi Bar,Portland,OR,4.5,31,"Sushi Bars, Restaurants, Poke, Food",45.471048,-122.648225,1,97202,b8wixAfPZpMs221Y-HBVLA
3,19909,EWp4zlEIL9uP0pt0SIxw4w,Teppanyaki Hut - North Portland,Portland,OR,4.5,86,"Sushi Bars, Ramen, Restaurants, Food Trucks, F...",45.554351,-122.675737,1,97217,EWp4zlEIL9uP0pt0SIxw4w
4,30448,XcEmmVORQ6ZXZ0TZHTr9_g,Krispy Rice,Portland,OR,3.0,18,"Sushi Bars, Food Delivery Services, Restaurant...",45.505603,-122.688215,1,97201,XcEmmVORQ6ZXZ0TZHTr9_g
5,94416,NGB3HCZYRr3Yu-KhgkqGUw,Meeka Japanese Restaurant,Portland,OR,4.0,213,"Sushi Bars, Restaurants, Japanese",45.527659,-122.475715,1,97230,NGB3HCZYRr3Yu-KhgkqGUw
6,87743,7u0jpnOLHLJUcE2nuHhVCw,SanSai Japanese Grill,Portland,OR,3.5,459,"Sushi Bars, Japanese, Restaurants",45.526845,-122.694592,1,97210,7u0jpnOLHLJUcE2nuHhVCw
7,119715,Hnf74SpjAIzgxLWIHgMBWw,Hissho Sushi,Portland,OR,3.5,69,"Sushi Bars, Restaurants, Japanese",45.587133,-122.592341,1,97218,Hnf74SpjAIzgxLWIHgMBWw
8,62612,2d7e7hkSvlBwlawrOWbP1w,Fujiyama Sushi Bar,Portland,OR,3.5,117,"Sushi Bars, Restaurants, Japanese",45.517932,-122.557398,1,97216,2d7e7hkSvlBwlawrOWbP1w
9,87910,NHAmk2FaoV-yyQFlUK2lRA,Bluefin Sushi Bar,Portland,OR,4.0,132,"Sushi Bars, Japanese, Restaurants",45.535002,-122.620576,1,97232,NHAmk2FaoV-yyQFlUK2lRA


In [56]:
name2[1:]

["Let's Roll PDX - Division",
 'Poke Box Sushi Bar',
 'Teppanyaki Hut - North Portland',
 'Krispy Rice',
 'Meeka Japanese Restaurant',
 'SanSai Japanese Grill',
 'Hissho Sushi',
 'Fujiyama Sushi Bar',
 'Bluefin Sushi Bar']

In [26]:
a.latitude.to_numpy()[0]

45.5124225

In [ ]:
.latitude.to_numpy()[0]
.longitude.to_numpy()[0]


In [29]:
data3=pd.read_csv('bagofWord.csv')

,bag_of_words
name,
10 Barrel Brewing Portland,dish flavor shop meat food service sweet beer ...
108 Bar N Grill,dish flavor shop meat food service bubble tea ...
10th Avenue Liquor Store,dish flavor salad meat food service saladbeer ...
11th Avenue Liquor,dish flavor tobacco shops meat food service sa...
13 Virtues Brewing,dish flavor shop meat salad food service sweet...
...,...
kBap,food service great ask peoplerestaurants peopl...
krazy cups and cafe,food service ask great definitelydesserts tabl...
red e café,flavor dish shop meat food service sweet great...


In [31]:
indices = pd.Series(data3.index)

In [32]:
indices

0          0
1          1
2          2
3          3
4          4
        ... 
3405    3405
3406    3406
3407    3407
3408    3408
3409    3409
Length: 3410, dtype: int64

In [35]:
idx = indices[indices == 'McMenamins Barley Mill Pub'].index[0]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [41]:
data=pd.read_csv('toronto_data2.csv')
# Combining the text in Keywords and categories columns
toronto_data=data
toronto_data['All_Keywords'] = toronto_data['categories'].str.cat(toronto_data['Keywords'],sep=", ")

# Formating the All_Keywords Column
toronto_data['All_Keywords'] = toronto_data['All_Keywords'].map(lambda x: str(x))
toronto_data['All_Keywords'] = toronto_data['All_Keywords'].map(lambda x: x.lower())

# Adding and Grouping Rows together by Restaurant Name
toronto_final = toronto_data.groupby('name')['All_Keywords'].sum()
toronto_final = toronto_final.to_frame(name = 'sum').reset_index()

# Getting a list of Unique Keywords per Restaurant

toronto_final['sum'] = toronto_final['sum'].map(lambda x: x.replace(", ","', '"))
toronto_final['sum'] = toronto_final['sum'].map(lambda x: str("'") + x + str("'"))
f = lambda x: x["sum"].split(", ")
toronto_final['sum'] = toronto_final.apply(f, axis=1)
toronto_final['sum'] = toronto_final['sum'].map(lambda x: set(x))
toronto_final.set_index('name', inplace = True)

# Creating Bag of Words
toronto_final['bag_of_words'] = ''
columns = toronto_final.columns
for index, row in toronto_final.iterrows():
    words = ''
    for col in columns:
            words = words + ' '.join(row[col])+ ' '
    row['bag_of_words'] = words
    
toronto_final.drop(columns = [col for col in toronto_final.columns if col!= 'bag_of_words'], inplace = True)

# Remove quotation marks
toronto_final['bag_of_words'] = toronto_final['bag_of_words'].map(lambda x: x.replace("'", ""))

In [42]:
indices = pd.Series(toronto_final.index)
print(indices)

0       10 Barrel Brewing Portland
1                  108 Bar N Grill
2         10th Avenue Liquor Store
3               11th Avenue Liquor
4               13 Virtues Brewing
                   ...            
3405                          kBap
3406           krazy cups and cafe
3407                    red e café
3408                      show bar
3409                   veggie bowl
Name: name, Length: 3410, dtype: object


In [43]:
toronto_final

,bag_of_words
name,
10 Barrel Brewing Portland,dish flavor shop meat food service sweet beer ...
108 Bar N Grill,dish flavor shop meat food service bubble tea ...
10th Avenue Liquor Store,dish flavor salad meat food service saladbeer ...
11th Avenue Liquor,dish flavor tobacco shops meat food service sa...
13 Virtues Brewing,dish flavor shop meat salad food service sweet...
...,...
kBap,food service great ask peoplerestaurants peopl...
krazy cups and cafe,food service ask great definitelydesserts tabl...
red e café,flavor dish shop meat food service sweet great...
